In [18]:
import ipynb.fs.full.google_search as gs
import ipynb.fs.full.gather_hl_and_articles as reader
import ipynb.fs.full.open_ai_models as openaimodel
import ipynb.fs.full.data_operations as do
import pandas as pd
import pprint as pp
import json

In [18]:
def main():

    analysts = [#'Sumeet Bagadia',
            #'Vaishali Parekh', 
            'Ganesh Dongre',
            'Dharmesh Shah',
            'Seema Srivastava',
            'Anshul Jain',
            'Shiju Koothupalakkal', 
            'Sugandha Sachdeva',
            'Mahesh M Ojha'
                ]
    
    
    ops = do.gspread_Operator()
    srch = gs.google_Search()
    llm = openaimodel.OpenAIModels()
    rdr = reader.Livemint_reader()
    
    #for loop should begin here:
    for analyst in analysts:
        try:
            print(f'Starting mass link search for {analyst}...')
            link_ids_already_seen = ops.read_specific_article_columns(['link_id'])
            #print(link_ids_already_seen.shape)
            google_links = srch.search(query=analyst, terms='Buy', num_pages=10, silent_mode=True)

            if(google_links.shape[0] == 0):
                break                              #no Google link was found for this analyst, so exit
            
            print(f'Found {google_links.shape[0]} links for {analyst} from Google for Livemint. Proceeding to dedupe against known links...')
            
            ##remove links we have previously seen
            seen_indices = google_links[google_links['link_id'].isin(link_ids_already_seen['link_id'])].index.tolist()
            if(len(seen_indices)>0):
                google_links.drop(index=seen_indices, inplace=True)
                print(f'Now remaining {google_links.shape[0]} links after dedupe...')
            
            response = llm.classify_headlines(google_links['title'], silent_mode=False)                                     # OpenAI's behavior is weird and it may or may not send the entire output or even the right output. 
                                                                                                                            # We are better off using Gemini, Or should we try 4.1 mini instead of nano?
            if (response != ''):
                output_json = json.loads(response)['output']
                classified_hl = pd.DataFrame(output_json)
                classified_titles = google_links.join(classified_hl, lsuffix='_orig', rsuffix='_copy')
                classified_titles = classified_titles[classified_titles['classification'] == True]                           #keeping only the titles with a analyst recommerndation here.
                classified_titles.drop_duplicates(subset=['link_id'], keep='first', inplace=True, ignore_index=False)        #remove duplicates that can creep in from multiple uls  
                #ops.write_headlines_in_chunks(classified_titles)
                for idx, row in classified_titles.iterrows():
                    article = rdr.read_article(row['link'])
                    if(not(article.empty)):
                        response2 = llm.gather_articles_susbtance(article, silent_mode=False)
                        if (response2 != ''):
                            recos = pd.DataFrame(json.loads(response2)['output'])
                            recos.insert(0, 'uuid', str(row['uuid']))
                            ops.write_recommendations(recos)
                            ops.write_headlines(row)
                            #break
                        else:
                            print(f'No recommendation could be gathered for {row['link']}...')
                    else:
                        print(f'No article could be retrieved for {row['link']}...') 
                    #break
                print(f'All articles read and compiled for {analyst}.')
            else:
                print(f'No classifications could be done for the articles...')
        except Exception as e:
            import traceback
            print(f'Failed execution with exception: {e}')
            traceback.print_exc()
        finally:
            continue
    
    return 0

In [5]:
if __name__ == '__main__':
    hl = main()

Traceback (most recent call last):
  File "/tmp/ipykernel_4955/3029552362.py", line 26, in main
    google_links = srch.search(query=analyst, terms='Buy', num_pages=10, silent_mode=True)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/fin_rec/analyse_analyst_recos/google_search.ipynb", line 57, in search
    "        return sanitized_query\n",
                                        
KeyError: 'og:description'
Traceback (most recent call last):
  File "/tmp/ipykernel_4955/3029552362.py", line 26, in main
    google_links = srch.search(query=analyst, terms='Buy', num_pages=10, silent_mode=True)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/fin_rec/analyse_analyst_recos/google_search.ipynb", line 57, in search
    "        return sanitized_query\n",
                                        
KeyError: 'og:description'
Traceback (most recent call last):
  File "/

In [3]:
import pandas
google_links2 = pandas.read_csv('links_found.csv', index_col=0)

In [4]:
google_links2.shape

(97, 5)

In [19]:
import ipynb.fs.full.open_ai_models as openaimodel

llm = openaimodel.OpenAIModels()
response = llm.classify_headlines(google_links2['title'], silent_mode=False)

Sending titles for classification to OpenAI model...
Input is of length: 97.


In [20]:
import json
import pprint as pp
output_json = json.loads(response)['output']
classified_hl = pandas.DataFrame(output_json)
classified_hl

,title,classification,explanation
0,Sumeet Bagadia recommended buying these seven ...,True,Contains explicit stock buy recommendation.
1,Stocks to buy under ₹100: Motherson Sumi Wiri...,True,Contains explicit stock buy suggestion under ₹...
2,Breakout stocks to buy or sell: Sumeet Bagadia...,True,"Mentions shares to buy today, clear recommenda..."
3,"Regarding stocks to buy under ₹100, Sumeet Ba...",True,Explicitly recommends buying specific stocks.
4,Buy or sell: Sumeet Bagadia recommends three s...,True,"Mentions stocks to buy, act on them."
...,...,...,...
91,Stocks to buy under ₹100: Sumeet Bagadia reco...,True,Explicit advice for stocks under ₹100.
92,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Explicit buy stocks today.
93,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Explicit stocks recommended to buy today.
94,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Explicit instruction to buy today.
